In [2]:
!pip install pydicom
!pip install scikit-learn
!pip install tqdm
!pip install scikit-learn
!pip install scikit-image
!pip install keras==2.1.5

  Attempting uninstall: keras
    Found existing installation: Keras 2.4.3
    Uninstalling Keras-2.4.3:
      Successfully uninstalled Keras-2.4.3


In [1]:
from ouassit import *

from keras.models import Model, load_model
from keras.layers import Input, BatchNormalization, Activation, Dense, Dropout
from keras.layers.convolutional import Conv2D, Conv2DTranspose, UpSampling2D
from keras.layers.pooling import MaxPooling2D, GlobalMaxPool2D
from keras.layers.merge import concatenate, add
from keras.layers.core import Lambda, RepeatVector, Reshape
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.utils import plot_model
from keras.optimizers import Adam
import matplotlib.pyplot as plt
#import pydot
import tensorflow as tf

from sklearn.model_selection import train_test_split

def check_pydot():
    try:
        # Attempt to create an image of a blank graph
        # to check the pydot/graphviz installation.
        pydot.Dot.create(pydot.Dot())
    except Exception:
        # pydot raises a generic Exception here,
        # so no specific class can be caught.
        raise ImportError('Failed to import pydot. You must install pydot'
                          ' and graphviz for `pydotprint` to work.')


#check_pydot()


ImportError: cannot import name 'K' from 'keras.models' (C:\ProgramData\Anaconda3\lib\site-packages\keras\models.py)

In [ ]:
import ouassit
from importlib import reload 
ouassit = reload(ouassit)
from ouassit import *

# Global variables

In [ ]:
home_path = "chaos"
image_width = 512
image_height = 512

# Load images

In [16]:
paths = [home_path]
def load_data():
    X, y = load_datasets(image_width, image_height, paths)
    return train_test_split(X, y, test_size=0.1, random_state=42)
X_train, X_valid, y_train, y_valid = load_data()

Looking in :  chaos/dicom Found :  2874
No. images :  2874
Loading No. images :  2874


# Create model

In [22]:
def Unet(nClasses, input_height, input_width):
    
    kernel_size = (3, 3)
    shape = (input_height, input_width, 1)
    
    inputs = Input(shape)
    
    # =============== encode ===============
    
    # layer 1
    conv1 = Conv2D(64, kernel_size, padding='same')(inputs)
    conv1 = BatchNormalization()(conv1)
    conv1 = (Activation('relu'))(conv1)
    conv1 = (Dropout(rate=1.0))(conv1)
    
    conv2 = Conv2D(64, kernel_size, padding='same')(conv1)
    conv2 = BatchNormalization()(conv2)
    conv2 = (Activation('relu'))(conv2)
    conv2 = (Dropout(rate=1.0))(conv2)
    
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv2)
    
    # layer 2
    conv3 = Conv2D(128, kernel_size, padding='same')(pool1)
    conv3 = BatchNormalization()(conv3)
    conv3 = (Activation('relu'))(conv3)
    conv3 = (Dropout(rate=1.0))(conv3)
    
    conv4 = Conv2D(128, kernel_size, padding='same')(conv3)
    conv4 = BatchNormalization()(conv4)
    conv4 = (Activation('relu'))(conv4)
    conv4 = (Dropout(rate=1.0))(conv4)
    
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv4)
    
    
    # layer 3
    conv5 = Conv2D(256, kernel_size, padding='same')(pool2)
    conv5 = BatchNormalization()(conv5)
    conv5 = (Activation('relu'))(conv5)
    conv5 = (Dropout(rate=1.0))(conv5)
    
    conv6 = Conv2D(256, kernel_size, padding='same')(conv5)
    conv6 = BatchNormalization()(conv6)
    conv6 = (Activation('relu'))(conv6)
    conv6 = (Dropout(rate=1.0))(conv6)
    
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv6)
    
    # layer 4
    conv7 = Conv2D(512, kernel_size, padding='same')(pool3)
    conv7 = BatchNormalization()(conv7)
    conv7 = (Activation('relu'))(conv7)
    conv7 = (Dropout(rate=1.0))(conv7)
    
    conv8 = Conv2D(512, kernel_size, padding='same')(conv7)
    conv8 = BatchNormalization()(conv8)
    conv8 = (Activation('relu'))(conv8)
    conv8 = (Dropout(rate=1.0))(conv8)
    
    pool4 = MaxPooling2D(pool_size=(2, 2))(conv8)

    # layer 5
    
    conv9 = Conv2D(1024, kernel_size, padding='same')(pool4)
    conv9 = BatchNormalization()(conv9)
    
    # =============== decode ===============
    
    o = (UpSampling2D((2, 2)))(conv9)
    o = (concatenate([o, conv8], axis=-1))
    o = (Conv2D(512, kernel_size, padding='same'))(o)
    o = (BatchNormalization())(o)
    o = (Activation('relu'))(o)
    o = (Dropout(rate=1.0))(o)
    
    o = (UpSampling2D((2, 2)))(o)
    o = (concatenate([o, conv6], axis=-1))
    o = (Conv2D(256, kernel_size, padding='same'))(o)
    o = (BatchNormalization())(o)
    o = (Activation('relu'))(o)
    o = (Dropout(rate=1.0))(o)
    
    o = (UpSampling2D((2, 2)))(o)
    o = (concatenate([o, conv4], axis=-1))
    o = (Conv2D(128, kernel_size, padding='same'))(o)
    o = (BatchNormalization())(o)
    o = (Activation('relu'))(o)
    o = (Dropout(rate=1.0))(o)

    o = (UpSampling2D((2, 2)))(o)
    o = (concatenate([o, conv2], axis=-1))
    o = (Conv2D(64, kernel_size, padding='same'))(o)
    o = (BatchNormalization())(o)
    o = (Activation('relu'))(o)
    o = (Dropout(rate=1.0))(o)
    
    o = (Conv2D(64, kernel_size, padding='same'))(o)
    o = (BatchNormalization())(o)
    o = (Activation('relu'))(o)
    o = (Dropout(rate=1.0))(o)
    

    o = Conv2D(nClasses, (1,1), padding='same')(o)
    outputs = Activation('softmax')(o)

    model = Model(inputs=[inputs], outputs=[outputs])
    return model

In [23]:
unet_model = Unet(1, image_height, image_width)

In [24]:

plot_model(unet_model, to_file='unet.png', show_shapes=True, show_layer_names=False)

ImportError: Failed to import pydot. You must install pydot and graphviz for `pydotprint` to work.

In [25]:
unet_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 512, 512, 1)  0                                            
__________________________________________________________________________________________________
conv2d_16 (Conv2D)              (None, 512, 512, 64) 640         input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_15 (BatchNo (None, 512, 512, 64) 256         conv2d_16[0][0]                  
__________________________________________________________________________________________________
activation_15 (Activation)      (None, 512, 512, 64) 0           batch_normalization_15[0][0]     
__________________________________________________________________________________________________
dropout_14

In [26]:
callbacks = [
    EarlyStopping(patience=10, verbose=1),
    ReduceLROnPlateau(factor=0.1, patience=5, min_lr=0.00001, verbose=1),
    ModelCheckpoint('unet-0.h5', verbose=1, save_best_only=True, save_weights_only=True)
]

unet_model.compile(optimizer=Adam(), loss="binary_crossentropy", metrics=["accuracy", dice_coef, dice_coef_loss, jaccard_index])


In [27]:
results = unet_model.fit(X_train, y_train, batch_size=16, epochs=50, callbacks=callbacks, validation_data=(X_valid, y_valid))

Train on 2586 samples, validate on 288 samples
Epoch 1/50
  64/2586 [..............................] - ETA: 6:29:10 - loss: 14.8342 - acc: 0.0693 - dice_coef: 0.1295 - dice_coef_loss: -0.1295 - jaccard_index: 0.0693

KeyboardInterrupt: 

In [ ]:
plt.figure(figsize=(8, 8))
plt.title("Learning curve")
plt.plot(results.history["loss"], label="loss")
plt.plot(results.history["val_loss"], label="val_loss")
plt.plot( np.argmin(results.history["val_loss"]), np.min(results.history["val_loss"]), marker="x", color="r", label="best model")
plt.xlabel("Epochs")
plt.ylabel("log_loss")
plt.legend()
plt.savefig('Learning_curve_loss.png')

In [ ]:
plt.figure(figsize=(8, 8))
plt.title("Learning curve")
plt.plot(results.history["accuracy"], label="accuracy")
plt.plot(results.history["val_accuracy"], label="val_accuracy")
plt.plot( np.argmin(results.history["val_accuracy"]), np.max(results.history["val_accuracy"]), marker="x", color="r", label="best model")
plt.xlabel("Epochs")
plt.ylabel("log_accuracy")
plt.legend();
plt.savefig('Learning_curve.png')

In [ ]:
plt.figure(figsize=(8, 8))
plt.title("Metrics")
plt.plot(results.history["dice_coef"], label="dice_coef")
plt.plot(results.history["dice_coef_loss"], label="dice_coef_loss")
plt.plot(results.history["jaccard_index"], label="jaccard_index")
plt.xlabel("Epochs")
plt.ylabel("log_acc")
plt.legend();
plt.savefig('metrics.png')

# CRF 

In [3]:
!pip install pydensecrf

  Using cached pydensecrf-1.0rc3.tar.gz (1.0 MB)
  Running setup.py clean for pydensecrf
Failed to build pydensecrf
    Running setup.py install for pydensecrf: started


  ERROR: Command errored out with exit status 1:
   command: 'C:\Users\jok4d\anaconda3\python.exe' -u -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'C:\\Users\\jok4d\\AppData\\Local\\Temp\\pip-install-7xdq7k9i\\pydensecrf_e05a7d3a1e05406998873fcba45e5591\\setup.py'"'"'; __file__='"'"'C:\\Users\\jok4d\\AppData\\Local\\Temp\\pip-install-7xdq7k9i\\pydensecrf_e05a7d3a1e05406998873fcba45e5591\\setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.exists(__file__) else io.StringIO('"'"'from setuptools import setup; setup()'"'"');code = f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' bdist_wheel -d 'C:\Users\jok4d\AppData\Local\Temp\pip-wheel-jssqs1tv'
       cwd: C:\Users\jok4d\AppData\Local\Temp\pip-install-7xdq7k9i\pydensecrf_e05a7d3a1e05406998873fcba45e5591\
  Complete output (79 lines):
  running bdist_wheel
  running build
  running build_py
  creating build
  creating build\lib.win-amd64-

    Running setup.py install for pydensecrf: finished with status 'error'



  C:\Users\jok4d\anaconda3\include\cpython/pystate.h(51): note: voir la déclaration de '_ts'
  pydensecrf/eigen.cpp(17130): error C2039: 'exc_traceback' n'est pas membre de '_ts'
  C:\Users\jok4d\anaconda3\include\cpython/pystate.h(51): note: voir la déclaration de '_ts'
  pydensecrf/eigen.cpp(17131): error C2039: 'exc_type' n'est pas membre de '_ts'
  C:\Users\jok4d\anaconda3\include\cpython/pystate.h(51): note: voir la déclaration de '_ts'
  pydensecrf/eigen.cpp(17132): error C2039: 'exc_value' n'est pas membre de '_ts'
  C:\Users\jok4d\anaconda3\include\cpython/pystate.h(51): note: voir la déclaration de '_ts'
  pydensecrf/eigen.cpp(17133): error C2039: 'exc_traceback' n'est pas membre de '_ts'
  C:\Users\jok4d\anaconda3\include\cpython/pystate.h(51): note: voir la déclaration de '_ts'
  error: command 'C:\\Program Files\\Microsoft Visual Studio\\2022\\Community\\VC\\Tools\\MSVC\\14.34.31933\\bin\\HostX86\\x64\\cl.exe' failed with exit code 2
  -------------------------------------

In [5]:
import numpy as np
import pydensecrf.densecrf as dcrf
from skimage.io import imread, imsave
from pydensecrf.utils import unary_from_labels, create_pairwise_bilateral
from skimage.color import gray2rgb
from skimage.color import rgb2gray
import matplotlib.pyplot as plt
import pandas as pd
from tqdm import tqdm
%matplotlib inline


ModuleNotFoundError: No module named 'pydensecrf'

In [ ]:
#Original_image = Image which has to labelled
#Mask image = Which has been labelled by some technique..
def crf(original_image, mask_img):
    
    # Converting annotated image to RGB if it is Gray scale
    if(len(mask_img.shape)<3):
        mask_img = gray2rgb(mask_img)

    ##Converting the annotations RGB color to single 32 bit integer
    annotated_label = mask_img[:,:,0] + (mask_img[:,:,1]<<8) + (mask_img[:,:,2]<<16)
    
    ## Convert the 32bit integer color to 0,1, 2, ... labels.
    colors, labels = np.unique(annotated_label, return_inverse=True)

    n_labels = 2
    
    #Setting up the CRF model
    d = dcrf.DenseCRF2D(original_image.shape[1], original_image.shape[0], n_labels)

    # get unary potentials (neg log probability)
    U = unary_from_labels(labels, n_labels, gt_prob=0.7, zero_unsure=False)
    d.setUnaryEnergy(U)

    # This adds the color-independent term, features are the locations only.
    d.addPairwiseGaussian(sxy=(3, 3), compat=3, kernel=dcrf.DIAG_KERNEL,
                      normalization=dcrf.NORMALIZE_SYMMETRIC)
        
    #Run Inference for 10 steps 
    Q = d.inference(10)

    # Find out the most probable class for each pixel.
    MAP = np.argmax(Q, axis=0)

    return MAP.reshape((original_image.shape[0],original_image.shape[1]))